In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [33]:
data = pd.read_csv('balance.csv')
data.head()

,first column,second column,Balance,Gender,Geography
0,1,10,2000,Male,India
1,2,20,2000,Female,France
2,3,30,40000,Male,Spain
3,4,40,57000,Female,India
4,1,10,7000,Male,India


In [34]:
## data preprocess data
## Drop irrelevent coloun
data = data.drop(['first column'],axis=1)
data

,second column,Balance,Gender,Geography
0,10,2000,Male,India
1,20,2000,Female,France
2,30,40000,Male,Spain
3,40,57000,Female,India
4,10,7000,Male,India
5,20,7000,Female,Us
6,30,80000,Male,Spain
7,40,97000,Female,India


In [39]:
## Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data

,second column,Balance,Gender,Geography
0,10,2000,1,India
1,20,2000,0,France
2,30,40000,1,Spain
3,40,57000,0,India
4,10,7000,1,India
5,20,7000,0,Us
6,30,80000,1,Spain
7,40,97000,0,India


In [41]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_India', 'Geography_Spain',
       'Geography_Us'], dtype=object)

In [43]:
## onehot encode geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo= OneHotEncoder()
geo_encoder= onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8 stored elements and shape (8, 4)>

In [44]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_India', 'Geography_Spain',
       'Geography_Us'], dtype=object)

In [46]:
# geo_encoder_df= pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
# geo_encoder_df
geo_encoder.toarray()

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [49]:
geo_encoder_df= pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_India,Geography_Spain,Geography_Us
0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,0.0,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0


In [50]:
## Combie one hot encoder column with orignal data
data =pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,second column,Balance,Gender,Geography_France,Geography_India,Geography_Spain,Geography_Us
0,10,2000,1,0.0,1.0,0.0,0.0
1,20,2000,0,1.0,0.0,0.0,0.0
2,30,40000,1,0.0,0.0,1.0,0.0
3,40,57000,0,0.0,1.0,0.0,0.0
4,10,7000,1,0.0,1.0,0.0,0.0


In [51]:
## save the encoder and sacaler
with open('label_encoder_gender.pk1','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pk1','wb') as file:
    pickle.dump(onehot_encoder_geo,file)    

In [58]:
X = data.drop('Balance', axis=1)
y = data['Balance']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[-1.33630621,  0.70710678,  0.        ,  0.70710678, -0.70710678,
         0.        ],
       [ 1.06904497, -1.41421356,  0.        ,  0.70710678, -0.70710678,
         0.        ],
       [ 0.26726124,  0.70710678,  0.        , -1.41421356,  1.41421356,
         0.        ],
       [-1.33630621,  0.70710678,  0.        ,  0.70710678, -0.70710678,
         0.        ],
       [ 1.06904497, -1.41421356,  0.        ,  0.70710678, -0.70710678,
         0.        ],
       [ 0.26726124,  0.70710678,  0.        , -1.41421356,  1.41421356,
         0.        ]])

In [60]:
with open('scaler.pk1','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [63]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime



In [65]:
X_train.shape[1]

6

In [68]:
## Build Our ANN Model
model = Sequential([
Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
Dense(32,activation='relu'),
Dense(1,activation='sigmoid') ##output layer

])

d:\python-project2\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,561 (10.00 KB)

 Trainable params: 2,561 (10.00 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x00000178B00D8FE0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [79]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])



In [81]:
## set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir='log/fit'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [82]:
## Set up Early Stoping
early_stopping_calback =EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)



In [84]:
## traning the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_calback]
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.0000e+00 - loss: -333720288.0000 - val_accuracy: 0.0000e+00 - val_loss: -11325272.0000
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.0000e+00 - loss: -346181408.0000 - val_accuracy: 0.0000e+00 - val_loss: -11736032.0000
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step - accuracy: 0.0000e+00 - loss: -358974816.0000 - val_accuracy: 0.0000e+00 - val_loss: -12157429.0000
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.0000e+00 - loss: -372105632.0000 - val_accuracy: 0.0000e+00 - val_loss: -12589618.0000
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.0000e+00 - loss: -385578848.0000 - val_accuracy: 0.0000e+00 - val_loss: -13032755.0000
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.0000e+00 - loss: -399399584.0000 - val_accuracy: 0.0000e+00 - val_loss: -13486999.0000
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.0000e+00 - loss: -41357

In [85]:
model.save('model.h5')

In [116]:
##load tensorboard extenstion
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [130]:
# %load_ext tensorboard
%tensorboard --logdir log/fit20250411-162905/
# %reload_ext tensorboard


Reusing TensorBoard on port 6010 (pid 8608), started 0:00:35 ago. (Use '!kill 8608' to kill it.)